In [14]:
import os
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM

In [17]:
mpl8_dir = 'local/'
drp = fits.open(mpl8_dir + 'drpall-v2_5_3.fits')
drpdata = drp[1].data 
stellar_mass = drpdata.field('nsa_sersic_mass')

In [18]:
# read in MPA-JHU catalog information
gal = fits.open(mpl8_dir + 'gal_info_dr7_v5_2.fit')
galdata = gal[1].data
sfr = fits.open(mpl8_dir + 'gal_totsfr_dr7_v5_2.fits')
sfrdata = sfr[1].data
mass = fits.open(mpl8_dir + 'gal_totsfr_dr7_v5_2.fits')
massdata = mass[1].data

In [19]:
# read in information from NSA catalog
nsa = fits.open(mpl8_dir + '1-nsa_v1_0_1.fits')
nsa_data = nsa[1].data

In [21]:
# match the MPA-JHU and MaNGA catalogs on the basis of RA and DEC
mpa_ra = galdata.field('RA')
mpa_dec = galdata.field('DEC')
bad = mpa_dec < -90
mpa_dec[bad] = -90
mpa_cat = SkyCoord(ra=mpa_ra*u.degree, dec=mpa_dec*u.degree)
idx_mpa, d2d_mpa, d3d_mpa = c.match_to_catalog_sky(mpa_cat) 

In [28]:
sfr_manga = sfrdata.field('MEDIAN')[idx_mpa]
re_arcsec = nsa_data.field('ELPETRO_TH50_R')
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
re_kpc = re_arcsec * u.arcsec / cosmo.arcsec_per_kpc_proper(z)
z = drpdata.field('nsa_z')
sigma_sfr = sfr / (2 * np.pi * re_kpc**2)

/Users/rileytheriault/anaconda3/lib/python3.7/site-packages/astropy/cosmology/core.py:1922: RuntimeWarning: invalid value encountered in sqrt
  return 2 * np.sqrt(x) * hyp2f1(1./6, 1./2, 7./6, -x**3)


ValueError: operands could not be broadcast together with shapes (641409,) (6779,) 